In [1]:
import or_gym
from or_gym.benchmarks.newsvendor.nv_benchmark import *
from or_gym.benchmarks.newsvendor.onv_benchmark import *
import matplotlib.pyplot as plt
plt.style.use('C:\ProgramData\Miniconda3\Lib\site-packages\matplotlib\style\orgym.py')
import pyomo.environ as pe

In [2]:
#create environments for 4 cases
env1_dfo=or_gym.make("NewsVendor-v2",env_config={'seed_int':20})
env1_mip=or_gym.make("NewsVendor-v2",env_config={'seed_int':20})
env1_mip_pi=or_gym.make("NewsVendor-v2")
env1_dfo_online=or_gym.make("NewsVendor-v2")
env1_mip_online=or_gym.make("NewsVendor-v2")

C:\ProgramData\Miniconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
#run optimizations for backlog scenario
env1_dfo_results = optimize_nv_dfo(env1_dfo)

env1_mip_model, env1_mip_results = optimize_nv_mip(env1_mip,
                                                   warmstart=True,
                                                   warmstart_kwargs={'mapping_env': env1_dfo,
                                                                     'mapping_z':env1_dfo_results.zopt})
#DFO
#reset env to run simulation with new demand profile and with base stock levels found
env1_dfo=or_gym.make("NewsVendor-v2")
#run simulation
for t in range(env1_dfo.num_periods):
    #take a step in the simulation using base stock policy
    env1_dfo.step(action=env1_dfo.base_stock_action(z=env1_dfo_results.zopt)) 

#MIP
zopt = list(env1_mip_model.z.get_values().values()) #extract optimal base stock levels
#reset env to run simulation with new demand profile and with base stock levels found
env1_mip=or_gym.make("NewsVendor-v2")
#run simulation
for t in range(env1_mip.num_periods):
    #take a step in the simulation using base stock policy
    env1_mip.step(action=env1_mip.base_stock_action(z=zopt)) 

   direc: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-3.83239377e+00,  7.02575134e+00,  2.42650276e+00],
       [-7.81588333e-04,  2.00000000e+00,  0.00000000e+00]])
    fopt: 0.4403902050025434
     fun: -0.43897082971336354
 message: 'Optimization terminated successfully.'
    nfev: 822
     nit: 10
  status: 0
 success: True
       x: array([ 87.00028331, 132.00001828,  82.47449046])
    xopt: array([ 87., 133.,  83.])
    zopt: array([ 87., 220., 303.])
Academic license - for non-commercial use only
Read LP format model from file C:\Users\HD\AppData\Local\Temp\tmp7uamvvmv.pyomo.lp
Reading time = 0.07 seconds
x3343: 4147 rows, 2980 columns, 13729 nonzeros
Read MIP start from file C:\Users\HD\AppData\Local\Temp\tmp0kt8h_yx.gurobi.mst
Optimize a model with 4147 rows, 2980 columns and 13729 nonzeros
Variable types: 1897 continuous, 1083 integer (1080 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+04]
  Objective range  [1e-05, 1e-03]
  Bounds ran

In [ ]:
NV_envs = {'DFO':env1_dfo,
           'DFO zopt':env1_dfo_results.zopt,
           'MIP':env1_mip,
           'MIP zopt':list(env1_mip_model.z.get_values().values())}
file_env = open('NV_envs_LS_part1.obj','wb')
pickle.dump(NV_envs,file_env)

In [ ]:
env1_mip_pi_model, env1_mip_pi_results = optimize_nv_pi_mip(env1_mip_pi,warmstart=True,
                                                   warmstart_kwargs={'mapping_env': env1_dfo,
                                                                     'mapping_z':env1_dfo_results.zopt,
                                                                     'perfect_information':True})

In [ ]:
NV_envs = {'Oracle':env1_mip_pi,
           'Oracle R':env1_mip_pi_model.R.get_values()}
file_env = open('NV_envs_LS_part2.obj','wb')
pickle.dump(NV_envs,file_env)

In [ ]:
_,_,env1_dfo_online_basestock = online_optimize_nv_dfo(env1_dfo_online)

In [ ]:
_,_,env1_mip_online_basestock = online_optimize_nv_mip(env1_mip_online,
                                                       solver_kwargs={'TimeLimit':3600},
                                                       warmstart=True)

In [ ]:
#plot results for backlog scenario

#profit
fig = plt.figure(figsize=[7,5])
plt.plot(np.cumsum(env1_dfo.P),label='DFO')
plt.plot(np.cumsum(list(env1_mip_model.P.get_values().values())),label='MIP')
plt.plot(np.cumsum(env1_dfo_online.P),label='oDFO')
plt.plot(np.cumsum(env1_mip_online.P),label='oMIP')
plt.plot(np.cumsum(env1_mip_pi.P),label='Oracle')
plt.xlabel('Day')
plt.ylabel('Cumulative Profit, $')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.savefig('NewsVendor_LostSales.png')
plt.show()

#base stock levels
N = env1_dfo.num_periods
M = env1_dfo.num_stages
env1_dfo_online_basestock = np.array(env1_dfo_online_basestock)
env1_mip_online_basestock = np.array(env1_mip_online_basestock)

def inventory_position(I,T,stage):
    return np.sum(I[:-1,:stage+1] + T[:-1,:stage+1],axis=1)

fig, axs = plt.subplots(3,1,figsize=[7,13])
axs = axs.ravel()
for i in range(M-1):
    axs[i].set_prop_cycle(None)
    axs[i].set_title('Echelon %i' %i)
    axs[i].set_xlabel('Day')

#     #plot base stock levels
#     axs[i].plot(env1_dfo_results.zopt[i]*np.ones(N),label='DFO BSL')
#     axs[i].plot(pe.value(env1_mip_model.z[i])*np.ones(N),label='MIP BSL')
#     axs[i].plot(np.append(env1_dfo_online.init_inv[i],env1_dfo_online_basestock[:,i]),label='oDFO BSL')
#     axs[i].plot(np.append(env1_mip_online.init_inv[i],env1_mip_online_basestock[:,i]),label='oMIP BSL')
    
    axs[i].set_prop_cycle(None)
    #plot actual inventory positions
    axs[i].plot(inventory_position(I = env1_dfo.I, T = env1_dfo.T, B = env1_dfo.B, stage = i),'--',label='DFO')
    axs[i].plot(inventory_position(I = env1_mip.I, T = env1_mip.T, B = env1_mip.B, stage = i),':',label='MIP')
    axs[i].plot(inventory_position(I = env1_dfo_online.I, T = env1_dfo_online.T, B = env1_dfo_online.B, stage = i),'-.',label='oDFO')
    axs[i].plot(inventory_position(I = env1_mip_online.I, T = env1_mip_online.T, B = env1_mip_online.B, stage = i),'-.',label='oMIP')
    axs[i].plot(inventory_position(I = env1_mip_pi.I, T = env1_mip_pi.T, B = env1_mip_pi.B, stage = i),label='Oracle')
    axs[i].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.savefig('NewsVendor_LostSales_inventory.png')
plt.show()

fig, axs = plt.subplots(3,1,figsize=[7,13])
axs = axs.ravel()
for i in range(M-1):
    axs[i].set_prop_cycle(None)
    axs[i].set_title('Echelon %i' %i)
    axs[i].set_xlabel('Day')

    #plot base stock levels
    axs[i].plot(env1_dfo_results.zopt[i]*np.ones(N),label='DFO')
    axs[i].plot(pe.value(env1_mip_model.z[i])*np.ones(N),label='MIP')
    axs[i].plot(np.append(env1_dfo_online.init_inv[i],env1_dfo_online_basestock[:,i]),label='oDFO')
    axs[i].plot(np.append(env1_mip_online.init_inv[i],env1_mip_online_basestock[:,i]),label='oMIP')
    
#     axs[i].set_prop_cycle(None)
#     #plot actual inventory positions
#     axs[i].plot(inventory_position(I = env1_dfo.I, T = env1_dfo.T, B = env1_dfo.B, stage = i),'--',label='DFO TIL')
#     axs[i].plot(inventory_position(I = env1_mip.I, T = env1_mip.T, B = env1_mip.B, stage = i),':',label='MIP TIL')
#     axs[i].plot(inventory_position(I = env1_dfo_online.I, T = env1_dfo_online.T, B = env1_dfo_online.B, stage = i),'-.',label='oDFO TIL')
#     axs[i].plot(inventory_position(I = env1_mip_online.I, T = env1_mip_online.T, B = env1_mip_online.B, stage = i),'-.',label='oMIP TIL')
#     axs[i].plot(inventory_position(I = env1_mip_pi.I, T = env1_mip_pi.T, B = env1_mip_pi.B, stage = i),'k',label='Oracle TIL')
    axs[i].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.savefig('NewsVendor_LostSales_zopt.png')
plt.show()
                
# #on hand inventory levels
# env1_dfo_online_basestock = np.array(env1_dfo_online_basestock)
# env1_mip_online_basestock = np.array(env1_mip_online_basestock)
# fig, axs = plt.subplots(3,1,figsize=[5,10])
# axs = axs.ravel()
# for i in range(M-1):
#     axs[i].set_title('Stage %i' %i)
#     axs[i].set_xlabel('Day')
#     axs[i].plot(env1_dfo.I[:,i],label='DFO')
#     axs[i].plot(np.reshape(list(env1_mip_model.I.get_values().values()),(N+1,env1_dfo.num_stages-1))[:,i],label='MIP')
#     axs[i].plot(env1_dfo_online.I[:,i],label='oDFO')
#     axs[i].plot(env1_mip_online.I[:,i],label='oMIP')
#     axs[i].legend()
# plt.tight_layout()
# plt.show()

In [ ]:
NV_envs = {'DFO':env1_dfo,
           'DFO zopt':env1_dfo_results.zopt,
           'MIP':env1_mip,
           'MIP zopt':list(env1_mip_model.z.get_values().values()),
           'Oracle':env1_mip_pi,
           'Oracle R':env1_mip_pi_model.R.get_values(),
           'oDFO':env1_dfo_online,
           'oDFO zopt':env1_dfo_online_basestock,
           'oMIP':env1_mip_online,
           'oMIP zopt':env1_mip_online_basestock}
file_env = open('NV_envs_LS.obj','wb')
pickle.dump(NV_envs,file_env)